## Домашнее задание № 2

1. Обработка пропусков - удаление пропусков, поскольку их не такое большое количество.

2. При обучении модели Doc2Vec на текстах меняем параметры: vector_size=10, а не 5.

3. Используется LinearRegression и Ridge.

In [68]:
# импорты необходимых библиотек
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
# %matplotlib inline

# import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [69]:
data = pd.read_csv('/content/IMDB-Movie-Data.csv')
print(data.shape)

data.head(3)

(1000, 12)


,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0


In [70]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Rank                1000 non-null   int64  
 1   Title               1000 non-null   object 
 2   Genre               1000 non-null   object 
 3   Description         1000 non-null   object 
 4   Director            1000 non-null   object 
 5   Actors              1000 non-null   object 
 6   Year                1000 non-null   int64  
 7   Runtime (Minutes)   1000 non-null   int64  
 8   Rating              1000 non-null   float64
 9   Votes               1000 non-null   int64  
 10  Revenue (Millions)  872 non-null    float64
 11  Metascore           936 non-null    float64
dtypes: float64(3), int64(4), object(5)
memory usage: 93.9+ KB


Видим, что есть нулевые данные в столбцах "Reveniue" и "Metascore". Удаляем строки с пропусками.


In [71]:
# удаляем пропуски
data = data.dropna()

# проверяем присутствие NaN
data.isna().any()

Rank                  False
Title                 False
Genre                 False
Description           False
Director              False
Actors                False
Year                  False
Runtime (Minutes)     False
Rating                False
Votes                 False
Revenue (Millions)    False
Metascore             False
dtype: bool

### Подготовка данных

Попробуем предсказывать рейтинг фильма по данным его описания, года, длины в минутах и кассовых сборов

Колонка "Rating" станет целевой переменной, или таргетом (y)
Остальных данные будут обучающей выборкой (X)


In [72]:
data.Description

0      A group of intergalactic criminals are forced ...
1      Following clues to the origin of mankind, a te...
2      Three girls are kidnapped by a man with a diag...
3      In a city of humanoid animals, a hustling thea...
4      A secret government agency recruits some of th...
                             ...                        
993    While still out to destroy the evil Umbrella C...
994    3 high school seniors throw a birthday party t...
996    Three American college students studying abroa...
997    Romantic sparks occur between two dance studen...
999    A stuffy businessman finds himself trapped ins...
Name: Description, Length: 838, dtype: object

In [73]:
# подготовим описания фильмов
# приведем все к нижнему регистру
data["text"] = data.Description.apply(lambda x: x.lower().split()) 

data["text"]

0      [a, group, of, intergalactic, criminals, are, ...
1      [following, clues, to, the, origin, of, mankin...
2      [three, girls, are, kidnapped, by, a, man, wit...
3      [in, a, city, of, humanoid, animals,, a, hustl...
4      [a, secret, government, agency, recruits, some...
                             ...                        
993    [while, still, out, to, destroy, the, evil, um...
994    [3, high, school, seniors, throw, a, birthday,...
996    [three, american, college, students, studying,...
997    [romantic, sparks, occur, between, two, dance,...
999    [a, stuffy, businessman, finds, himself, trapp...
Name: text, Length: 838, dtype: object

In [74]:
#data.text.values

input_text = list(data.text.values)

In [75]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(input_text)]
documents[10:12]

[TaggedDocument(words=['the', 'story', 'of', 'a', 'team', 'of', 'female', 'african-american', 'mathematicians', 'who', 'served', 'a', 'vital', 'role', 'in', 'nasa', 'during', 'the', 'early', 'years', 'of', 'the', 'u.s.', 'space', 'program.'], tags=[10]),
 TaggedDocument(words=['the', 'rebel', 'alliance', 'makes', 'a', 'risky', 'move', 'to', 'steal', 'the', 'plans', 'for', 'the', 'death', 'star,', 'setting', 'up', 'the', 'epic', 'saga', 'to', 'follow.'], tags=[11])]

Обучаем модель на текстах описаний фильмов. Изменены параметры. Вместо vector_size=5 берем vectro_size=10. Изменение других параметров к более хорошим вариантам не привели.

In [77]:
model = Doc2Vec(documents, vector_size=10, window=2, min_count=1, workers=4)

In [78]:
model.save("D2V.model") # сохранение модели

In [79]:
# так можно посмотреть на векторы текстов, на которых училась модель
# индекс [] около documents -- это индекс текста из датасета

model.dv[documents[0].tags[0]]

array([-0.01227106,  0.01687276, -0.06253909,  0.08252369,  0.07383683,
        0.07122044,  0.08746436,  0.07678494, -0.3332221 , -0.07985588],
      dtype=float32)

Теперь нужно добавить векторы в датасет с остальными параметрами

In [80]:
# создадим список с векторами для каждого текста
vectors = []
for x in documents:
    vec = list(model.dv[x.tags][0])
    vectors.append(vec)

In [81]:
# так получим датафрейм, где все компоненты векторов в отдельных столбцах
split_df = pd.DataFrame(vectors,
                        columns=['v1', 'v2', 'v3','v4',"v5", 'v6', 'v7', 'v8', 'v9', 'v10'])

split_df

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10
0,-0.012271,0.016873,-0.062539,0.082524,0.073837,0.071220,0.087464,0.076785,-0.333222,-0.079856
1,0.060795,0.102217,-0.015932,-0.031301,-0.002146,-0.035142,0.169377,0.194300,-0.277242,-0.109442
2,0.019787,0.147390,-0.005479,0.027772,0.120723,0.024080,0.202464,0.097695,-0.315838,-0.248570
3,0.123063,0.196499,-0.016807,-0.048244,0.054565,0.121980,0.284362,0.114921,-0.421155,-0.145116
4,0.028230,0.031464,0.085937,-0.088259,0.087351,0.041308,0.261676,0.153166,-0.264149,-0.213961
...,...,...,...,...,...,...,...,...,...,...
833,-0.003788,0.207584,0.138312,0.003274,0.162514,0.078346,0.432025,0.259848,-0.417181,-0.223352
834,0.162929,0.110740,0.096853,-0.001289,0.086349,0.098972,0.398449,0.203325,-0.488422,-0.228388
835,0.042628,0.086123,0.108166,-0.005595,0.125860,0.130880,0.175750,0.165954,-0.378185,-0.097846
836,0.065441,0.120075,0.043109,0.047875,-0.045167,0.056735,0.154178,0.196909,-0.202333,-0.004620


In [82]:
# теперь добавим его к основному датафрейму
result = data.join(split_df, how='left')
result.shape

(838, 23)

In [83]:
result = result.dropna()
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 0 to 837
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Rank                712 non-null    int64  
 1   Title               712 non-null    object 
 2   Genre               712 non-null    object 
 3   Description         712 non-null    object 
 4   Director            712 non-null    object 
 5   Actors              712 non-null    object 
 6   Year                712 non-null    int64  
 7   Runtime (Minutes)   712 non-null    int64  
 8   Rating              712 non-null    float64
 9   Votes               712 non-null    int64  
 10  Revenue (Millions)  712 non-null    float64
 11  Metascore           712 non-null    float64
 12  text                712 non-null    object 
 13  v1                  712 non-null    float32
 14  v2                  712 non-null    float32
 15  v3                  712 non-null    float32
 16  v4      

In [84]:
# переопределим датасет, оставив только важное

data_sm = result[['Runtime (Minutes)',"Year",
                'Rating', 'Votes',
                'Revenue (Millions)','Metascore',"v1","v2","v3","v4","v5", 'v6', 'v7', 'v8', 'v9', 'v10']
              ]


data_sm.head(3)

,Runtime (Minutes),Year,Rating,Votes,Revenue (Millions),Metascore,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10
0,121,2014,8.1,757074,333.13,76.0,-0.012271,0.016873,-0.062539,0.082524,0.073837,0.071220,0.087464,0.076785,-0.333222,-0.079856
1,124,2012,7.0,485820,126.46,65.0,0.060795,0.102217,-0.015932,-0.031301,-0.002146,-0.035142,0.169377,0.194300,-0.277242,-0.109442
2,117,2016,7.3,157606,138.12,62.0,0.019787,0.147390,-0.005479,0.027772,0.120723,0.024080,0.202464,0.097695,-0.315838,-0.248570


### Подготавливаем матрицы

In [85]:
# определяем X и y

X = data_sm.drop(["Rating"],axis=1).values 

display(X, X.shape)

array([[ 1.21000000e+02,  2.01400000e+03,  7.57074000e+05, ...,
         7.67849386e-02, -3.33222091e-01, -7.98558816e-02],
       [ 1.24000000e+02,  2.01200000e+03,  4.85820000e+05, ...,
         1.94300041e-01, -2.77241707e-01, -1.09441757e-01],
       [ 1.17000000e+02,  2.01600000e+03,  1.57606000e+05, ...,
         9.76945907e-02, -3.15837830e-01, -2.48570070e-01],
       ...,
       [ 1.08000000e+02,  2.01400000e+03,  3.88040000e+04, ...,
         1.65954232e-01, -3.78184855e-01, -9.78456214e-02],
       [ 1.28000000e+02,  2.01600000e+03,  5.53100000e+03, ...,
         1.96909323e-01, -2.02333316e-01, -4.62046172e-03],
       [ 1.13000000e+02,  2.00800000e+03,  1.63144000e+05, ...,
         2.05769718e-01, -1.37473911e-01, -1.19573258e-01]])

(712, 15)

In [86]:
data_sm.isna().any()

Runtime (Minutes)     False
Year                  False
Rating                False
Votes                 False
Revenue (Millions)    False
Metascore             False
v1                    False
v2                    False
v3                    False
v4                    False
v5                    False
v6                    False
v7                    False
v8                    False
v9                    False
v10                   False
dtype: bool

In [87]:
y = data_sm['Rating'].values # отдельно вынесли массив со значениями скорости ветра
y.shape

(712,)

Иногда бывает полезно нормализовать данные: это позволяет исправить ситуацию, когда признаки представлены в разных единацах измерения. Для этого используется StandardScaler.

До нормализации:

In [88]:
list(X[0])

[121.0,
 2014.0,
 757074.0,
 333.13,
 76.0,
 -0.012271064333617687,
 0.016872763633728027,
 -0.06253909319639206,
 0.08252368867397308,
 0.07383682578802109,
 0.07122044265270233,
 0.0874643623828888,
 0.07678493857383728,
 -0.33322209119796753,
 -0.07985588163137436]

In [89]:
# используем стандартизатор
sc = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(sc.fit_transform(X), y, random_state=42)

После:

In [90]:
list(sc.fit_transform(X)[0])

[0.291007552571261,
 0.4090342348627316,
 2.7018134034374643,
 2.2071532994957423,
 0.9140238266037984,
 -1.095142079352365,
 -1.4882140068536893,
 -1.793751563307469,
 1.4304787577540878,
 0.0921426249929003,
 -0.5338904475206026,
 -1.67489872346434,
 -1.3368648162147279,
 0.31189319025887535,
 0.9720200956378233]

In [92]:
# задаем модель регрессора
# силу регуляризации можно варьировать параметром alpha
regressor_1 = LinearRegression()
regressor_2 = Ridge() # варьирование параметра alpha значение метрик не улучшило

# обучаем
regressor_1.fit(X_train, y_train)
regressor_2.fit(X_train, y_train)

Ridge()

In [93]:
# давайте предскажем результат для тестовой выборки

y_1_preds = regressor_1.predict(X_test)
y_2_preds = regressor_2.predict(X_test)


### оценка результатов алгоритма
В качестве метрики будем использовать среднюю абсолютную ошибку. Она показывает отклонение от правильного ответа в тех же единах измерения

(а вообще есть разные способы)

In [97]:
print('MAE модели LinearRegression:')
print(mean_absolute_error(y_test, y_1_preds))
print('')
print('MAE модели Ridge:')
print(mean_absolute_error(y_test, y_2_preds))
print('')
print('MSE модели LinearRegression:')
print(mean_squared_error(y_test, y_1_preds))
print('')
print('MSE модели Ridge:')
print(mean_squared_error(y_test, y_2_preds))

MAE модели LinearRegression:
0.42440862286440756

MAE модели Ridge:
0.42437663765266387
 
MSE модели LinearRegression:
0.3302016554783566

MSE модели Ridge:
0.3300933565659015


### Вывод:

Как видим лучшее значение показывает модель Ridge.